In [236]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix,accuracy_score


In [237]:
#read data
X_train = pd.read_csv('/Users/shreya/Downloads/X_Train_SMOTE.csv')
y_train = pd.read_csv('/Users/shreya/Downloads/y_train_SMOTE.csv')
X_test = pd.read_csv('/Users/shreya/Desktop/STAT 503/test_x.csv')
y_test = pd.read_csv('/Users/shreya/Desktop/STAT 503/test_labels.csv')

In [238]:
y_train = np.ravel(y_train)
y_test = np.ravel(y_test)

## Classifier 1

**Binary target is 1 if fibrosis level > 0; classifer predicts Pr(Target>0)**

In [224]:
# Change y_train to binary
y_train1=y_train
y_train1=np.where(y_train==0, 0, 1)


#initiate and tune
model1 = RandomForestClassifier()

model1.fit(X_train,y_train1)

param_grid = {
                'n_estimators': [5, 10, 20, 30, 40, 50],
                'max_depth': [2, 7, 12, 17],
                'criterion': ["entropy"],
                'random_state': [0],
                'min_samples_split': [8, 10, 12],
                'min_samples_leaf': [3, 4, 5],
                'bootstrap': [True],
                'random_state': [0]
    
             }

grid_clf1 = GridSearchCV(model1, param_grid, cv=5, n_jobs=-1)
grid_clf1.fit(X_train,y_train1)

#best model
m1=grid_clf1.best_estimator_

## Classifier 2

**Binary target is 1 fibrosis level > 1, so the classifier will predict Pr(Target>1)**

In [332]:
# Change y_train to binary
y_train2=y_train
y_train2=np.where((y_train2==0)|(y_train2==1), 0, 1)

from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(hidden_layer_sizes=(200,500,4),
                    random_state=0,
                    activation='relu',
                    solver='sgd',
                    batch_size=10)

# Fit data onto the model
clf.fit(X_train.values,y_train2)
m2=clf

## Classifier 3

**Binary target is 1 fibrosis level > 2, so the classifier will predict Pr(Target>2)**

In [317]:
from sklearn.svm import SVC

y_train3=y_train
y_train3=np.where((y_train3==0)|(y_train3==1)|(y_train3==2), 0, 1)

svm = SVC(random_state=0, kernel='rbf', probability=True)

# Fit the data to the SVM classifier
svm = svm.fit(X_train, y_train3)

m3=svm

In [333]:
pred1=m1.predict_proba(X_test)
pred2=m2.predict_proba(X_test.values)
pred3=m3.predict_proba(X_test)

In [334]:
pred_1_proba=pred1[:,1]
pred_2_proba=pred2[:,1]
pred_3_proba=pred3[:,1]

BD=1-pred_1_proba  #blood donor
H=pred_1_proba - pred_2_proba #hepatitis
F=pred_2_proba-pred_3_proba #Fibrosis
C=pred_3_proba #cirrhosis

In [335]:
merged_df=pd.DataFrame(np.column_stack((BD,H,F,C)), columns=[0, 1, 2, 3])
merged_df['predicted'] = merged_df.idxmax(axis=1)

In [336]:
cm=confusion_matrix(y_test,merged_df['predicted'])
cm

array([[160,   0,   1,   1],
       [  1,   3,   2,   1],
       [  0,   1,   4,   1],
       [  0,   1,   2,   6]])

In [344]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test,merged_df['predicted'])
score

0.9402173913043478

In [337]:
from sklearn.metrics import recall_score
recall_score(y_test,merged_df['predicted'], average=None)

array([0.98765432, 0.42857143, 0.66666667, 0.66666667])

In [338]:
from sklearn.metrics import precision_score
precision_score(y_test,merged_df['predicted'], average=None)

array([0.99378882, 0.6       , 0.44444444, 0.66666667])

In [341]:
import sklearn.metrics 
sklearn.metrics.f1_score(y_test,merged_df['predicted'], average=None)

array([0.99071207, 0.5       , 0.53333333, 0.66666667])